In [4]:
import pandas as pd
import numpy as np
from keras.layers import Input, LSTM, Concatenate, Dense
from keras.models import Model
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [52]:
# data preparation
raw_data = pd.read_csv("/content/drive/My Drive/GOOG-year.csv")
#raw_data =raw_data[:50] # Juml row maksimal yang mau diambil sample 15 row

# memilih kolom open dan close saja
raw_data = raw_data[['Open', 'Close','High','Low','Adj Close']] # all = Tanggal,Open,High,Low,Close,Adj Close,Volume
raw_data.head()

,Open,Close,High,Low,Adj Close
0,778.200012,768.700012,781.650024,763.450012,768.700012
1,767.250000,762.130005,769.950012,759.030029,762.130005
2,750.659973,762.020020,770.359985,750.560974,762.020020
3,774.500000,782.520020,785.190002,772.549988,782.520020
4,783.400024,790.510010,795.632996,780.190002,790.510010


In [50]:
data = raw_data[['Open']]
scaler = MinMaxScaler(feature_range=(0, 1))
open = scaler.fit_transform(data)

data = raw_data[['Close']]
scaler = MinMaxScaler(feature_range=(0, 1))
close = scaler.fit_transform(data)

data = raw_data[['High']]
scaler = MinMaxScaler(feature_range=(0, 1))
high = scaler.fit_transform(data)

data = raw_data[['Low']]
scaler = MinMaxScaler(feature_range=(0, 1))
low = scaler.fit_transform(data)

data = raw_data[['Adj Close']]
scaler = MinMaxScaler(feature_range=(0, 1))
adj = scaler.fit_transform(data)



In [88]:
# membuat dataset dengan 90% data training dan 10% data testing
train_size = int(len(open) * 0.5) #226 total 252

#Datase Test data 90%
train_data = open[:train_size] # 90% = 226 -252 = 26  /Train 90%
original_array=np.array(train_data) #trainX & trainY
open = original_array.ravel()   

#Datase Test data 90%
train_data = close[:train_size] # 90% = 226 -252 = 26  /Train 90%
original_array=np.array(train_data) #trainX & trainY
close = original_array.ravel()   

#Datase Test data 90%
train_data = high[:train_size] # 90% = 226 -252 = 26  /Train 90%
original_array=np.array(train_data) #trainX & trainY
high = original_array.ravel()   

#Datase Test data 90%
train_data = low[:train_size] # 90% = 226 -252 = 26  /Train 90%
original_array=np.array(train_data) #trainX & trainY
low = original_array.ravel()   

#Datase Test data 90%
train_data = adj[:train_size] # 90% = 226 -252 = 26  /Train 90%
original_array=np.array(train_data) #trainX & trainY
adj = original_array.ravel()   

#Datase Test data 90%
train_data = open[:train_size] # 90% = 226 -252 = 26  /Train 90%
original_array=np.array(train_data) #trainX & trainY
open = original_array.ravel()   


#Datase Test data 10%
test_data = close[train_size:]       #/ Test 10%
original_array1=np.array(test_data) #testX & testY
test_data = original_array1.ravel()

#reshape data ke 3D
open_data = open.reshape((len(open), 1,1))
close_data = close.reshape((len(close),1, 1))
high_data = high.reshape((len(high), 1,1))
low_data = low.reshape((len(low), 1,1))
volume_data = adj.reshape((len(adj),1, 1))

In [54]:
#input layer / Setting variable jadi 3 dimensi
open_in = Input(shape=(1,1))
close_in = Input(shape=(1,1))
high_in = Input(shape=(1,1))
low_in = Input(shape=(1,1))
volume_in = Input(shape=(1,1))

#LSTM layer
open_lstm = LSTM(50)(open_in)
close_lstm = LSTM(50)(close_in)
high_lstm = LSTM(50)(high_in)
low_lstm = LSTM(50)(low_in)
volume_lstm = LSTM(50)(volume_in)

In [89]:
#concatenate layer
merged = Concatenate()([open_lstm, close_lstm, high_lstm, low_lstm, volume_lstm])

#output layer
output = Dense(1)(merged)

#model
model = Model([open_in, close_in, high_in, low_in, volume_in], output)
model.compile(loss='mean_absolute_error', optimizer='adam')

#fit model
model.fit([open_data, close_data, high_data, low_data, volume_data], open_data, epochs=3, batch_size=32)
#model.fit([open_data, close_data, high_data, low_data, volume_data], data['Next_Close'], epochs=5, batch_size=32) # buatkan datanya

Epoch 1/3
2/2 [==============================] - 9s 12ms/step - loss: 0.1179
Epoch 2/3
2/2 [==============================] - 0s 15ms/step - loss: 0.1005
Epoch 3/3
2/2 [==============================] - 0s 12ms/step - loss: 0.0836


In [ ]:
#predict
predictions = model.predict([open_data, close_data, high_data, low_data, volume_data])

#evaluate
error = model.evaluate([open_data, close_data, high_data, low_data, volume_data], open_data)   #error = model.evaluate([open_data, close_data, high_data, low_data, volume_data], data['Close'])
print("Absolute Mean Error: ", error)
#train_data1=train_data

# inverting data kembali ke nilai asli
predictions = scaler.inverse_transform(predictions) #2D
train_data=train_data.reshape(-1, 1) #ubah ke 2D
train_data   = scaler.inverse_transform(train_data) # cuma sekali presiksi saja karean bisa ke scaller 2 x

#test_data   = scaler.inverse_transform(test_data)
#testPredict = scaler.inverse_transform(testPredict)                       
#testY = scaler.inverse_transform(testY)     

#plot
plt.plot(predictions, label='Predictions')
#plt.plot(data['Next_Close'], label='Actual')
#plt.plot(new_array1, label='Actual') # buatkan datanya
plt.plot(train_data, label='Hasil Training') # buatkan datanya
plt.legend()
plt.show()